# One Runway

## **Single-Runway Constraint Programming Model**

<h4 style="color:#0033a0">Parameters</h4>




\begin{align}
p & \quad \text{Number of aircraft} \\
E_i & \quad \text{Earliest landing time allowed for aircraft } i \\
T_i & \quad \text{Target (preferred) landing time for aircraft } i \\
L_i & \quad \text{Latest landing time allowed for aircraft } i \\
e_i & \quad \text{Penalty cost per unit time if } t_i < T_i \ (\text{earliness cost}) \\
l_i & \quad \text{Penalty cost per unit time if } t_i > T_i \ (\text{lateness cost}) \\
S_{i,j} & \quad \text{Separation time needed if aircraft } i \text{ lands immediately before } j \\
\end{align}

<br/> <h4 style="color:#0033a0">Decision Variables</h4>

\begin{align*}
t_i & \quad \text{Landing time of aircraft } i \\
E'_i & \quad \text{Earliness of aircraft } i \quad (E'_i \ge 0) \\
L'_i & \quad \text{Lateness of aircraft } i \quad (L'_i \ge 0) \\
x_{i,j} & =
\begin{cases}
1, & \text{if } i \text{ lands before } j,\\
0, & \text{otherwise}
\end{cases}
\end{align*}


<br/> <h4 style="color:#0033a0">Objective Function and Constraints</h4>

\begin{align}
\text{Minimize} \quad
& \sum_{i=1}^p \Bigl( e_i \, E'_i \;+\; l_i \, L'_i \Bigr)
&& (1)
\\[6pt]
\text{subject to:} \quad
& E_i \;\le\; t_i \;\le\; L_i
&& \forall i
&& (2)
\\
& t_i + E'_i = T_i 
&& \forall i \text{ with } t_i \le T_i 
&& (3)
\\
& t_i - L'_i = T_i
&& \forall i \text{ with } t_i \ge T_i
&& (4)
\\
& t_j \;\ge\; t_i \;+\; S_{i,j} \;-\; M \,(1 - x_{i,j})
&& \forall i \neq j
&& (5)
\\
& t_i \;\ge\; t_j \;+\; S_{j,i} \;-\; M \, x_{i,j}
&& \forall i \neq j
&& (6)
\\
& x_{i,j} + x_{j,i} = 1
&& \forall i \neq j
&& (7)
\\
& x_{i,j} \in \{0,1\},\; E'_i, L'_i \ge 0,\; t_i \ge 0
&& (8)
\end{align}


In [1]:
def read_data_from_file(filename):
    """
    Reads data from a file with the specified format for an air traffic scheduling problem.

    Args:
        filename (str): The path to the data file.

    Returns:
        tuple: A tuple containing the parsed data:
            - num_planes (int): The number of planes.
            - freeze_time (int): The freeze time.
            - planes_data (list): A list of dictionaries, where each dictionary contains
              the data for a plane.
            - separation_times (list of lists): A 2D list of separation times.
    """
    try:
        with open(filename, "r") as f:
            # Read the first line: number of planes and freeze time
            first_line = f.readline().strip().split()
            num_planes = int(first_line[0])
            freeze_time = int(first_line[1])

            planes_data = []
            separation_times = []
            
            for _ in range(num_planes):
                line = f.readline().strip().split()
                appearance_time = int(line[0])
                earliest_landing_time = int(line[1])
                target_landing_time = int(line[2])
                latest_landing_time = int(line[3])
                penalty_early = float(line[4])
                penalty_late = float(line[5])
                planes_data.append(
                    {
                        "appearance_time": appearance_time,
                        "earliest_landing_time": earliest_landing_time,
                        "target_landing_time": target_landing_time,
                        "latest_landing_time": latest_landing_time,
                        "penalty_early": penalty_early,
                        "penalty_late": penalty_late,
                    }
                )
                
                separation_row = []
                while len(separation_row) < num_planes:
                    line = f.readline().strip().split()
                    separation_row.extend([int(x) for x in line])

                separation_times.append(separation_row)

        return num_planes, freeze_time, planes_data, separation_times

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None, None, None, None
    except ValueError:
        print(f"Error: Error reading data in file '{filename}'.")
        return None, None, None, None

In [2]:
filename = "data/airland1.txt"

num_planes, freeze_time, planes_data, separation_times = read_data_from_file(filename)

if num_planes is not None:
    print("Number of planes:", num_planes)
    print("Freeze time:", freeze_time)
    print("\nPlane data:")
    for i, plane in enumerate(planes_data):
        print(f"Plane {i+1}: {plane}")
    print("\nSeparation times:")
    for i, row in enumerate(separation_times):
        print(f"After plane {i+1}: {row}")

Number of planes: 10
Freeze time: 10

Plane data:
Plane 1: {'appearance_time': 54, 'earliest_landing_time': 129, 'target_landing_time': 155, 'latest_landing_time': 559, 'penalty_early': 10.0, 'penalty_late': 10.0}
Plane 2: {'appearance_time': 120, 'earliest_landing_time': 195, 'target_landing_time': 258, 'latest_landing_time': 744, 'penalty_early': 10.0, 'penalty_late': 10.0}
Plane 3: {'appearance_time': 14, 'earliest_landing_time': 89, 'target_landing_time': 98, 'latest_landing_time': 510, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 4: {'appearance_time': 21, 'earliest_landing_time': 96, 'target_landing_time': 106, 'latest_landing_time': 521, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 5: {'appearance_time': 35, 'earliest_landing_time': 110, 'target_landing_time': 123, 'latest_landing_time': 555, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 6: {'appearance_time': 45, 'earliest_landing_time': 120, 'target_landing_time': 135, 'latest_landing_time': 576, 'penalty_earl

In [3]:
from ortools.sat.python import cp_model


def create_or_tools_cp_model(num_planes, freeze_time, planes_data, separation_times):
    """
    Creates an OR-Tools CP-SAT model (and variables) for the single-runway 
    aircraft landing problem with earliness/lateness, using the same data 
    structure as the MIP example.

    Args:
        num_planes (int): Number of planes.
        freeze_time (int): Freeze time (not explicitly used here, unless you wish to).
        planes_data (list[dict]): For each plane i:
            {
                'appearance_time': ...,
                'earliest_landing_time': ...,
                'target_landing_time': ...,
                'latest_landing_time': ...,
                'penalty_early': ...,
                'penalty_late': ...
            }
        separation_times (list[list]): S[i][j] = min. separation if plane i lands before j.

    Returns:
        model (cp_model.CpModel): The CP-SAT model object.
        variables (dict): A dictionary with keys:
            - "landing_times": list of IntVar for each plane's landing time.
            - "earliness": list of IntVar for each plane's earliness.
            - "lateness": list of IntVar for each plane's lateness.
            - "landing_order": dict of BoolVar x[(i,j)] for ordering (i before j).
    """
    model = cp_model.CpModel()

    # Large constant for "big-M"-style constraints (we'll do reified constraints below).
    # You could also do direct disjunction constraints, but we'll keep it consistent
    # with the MIP approach, using x[i,j].
    M = 10_000_000

    # Extract data in simpler arrays
    earliest = [p["earliest_landing_time"] for p in planes_data]
    target   = [p["target_landing_time"]   for p in planes_data]
    latest   = [p["latest_landing_time"]   for p in planes_data]
    cost_e   = [p["penalty_early"]         for p in planes_data]  # e_i
    cost_l   = [p["penalty_late"]          for p in planes_data]  # l_i

    # 1) Decision Variables

    # Landing times: integer vars in [0, large bound]
    landing_times = [
        model.NewIntVar(0, 10_000_000, f"landing_time_{i}") for i in range(num_planes)
    ]

    # Earliness and lateness
    earliness = [
        model.NewIntVar(0, 10_000_000, f"earliness_{i}") for i in range(num_planes)
    ]
    lateness = [
        model.NewIntVar(0, 10_000_000, f"lateness_{i}") for i in range(num_planes)
    ]

    # Binary ordering variables: x[i,j] = 1 if plane i lands before plane j
    landing_order = {}
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                landing_order[(i, j)] = model.NewBoolVar(f"LandingOrder_{i}_{j}")

    # 2) Constraints

    # (a) Time windows: earliest <= landing_times[i] <= latest
    for i in range(num_planes):
        model.Add(landing_times[i] >= earliest[i])
        model.Add(landing_times[i] <= latest[i])

    # (b) Earliness / Lateness relation:
    #     earliness[i] = max(0, target[i] - landing_times[i])
    #     lateness[i]  = max(0, landing_times[i] - target[i])
    # We can encode each piecewise simply as:
    #
    #   earliness[i] >= target[i] - landing_times[i]
    #   earliness[i] >= 0
    #   lateness[i]  >= landing_times[i] - target[i]
    #   lateness[i]  >= 0
    #
    # This is a common "big-V" approach for earliness/lateness in CP:
    for i in range(num_planes):
        model.Add(earliness[i] >= target[i] - landing_times[i])
        model.Add(earliness[i] >= 0)
        model.Add(lateness[i]  >= landing_times[i] - target[i])
        model.Add(lateness[i]  >= 0)

    # (c) Separation constraints:
    # If x[i,j] = 1 => plane j can't land until plane i + S[i][j]
    # If x[i,j] = 0 => plane i can't land until plane j + S[j][i]
    #
    # We'll use OR-Tools "reified" approach:
    #
    #   landing_times[j] >= landing_times[i] + S[i][j] - M*(1 - x[i,j])
    #   landing_times[i] >= landing_times[j] + S[j][i] - M*x[i,j]
    #
    # But a more CP-typical approach in OR-Tools is to use OnlyEnforceIf:
    #   model.Add(landing_times[j] >= landing_times[i] + separation_times[i][j]).OnlyEnforceIf(landing_order[i,j])
    #   model.Add(landing_times[i] >= landing_times[j] + separation_times[j][i]).OnlyEnforceIf(landing_order[i,j].Not())
    #
    # We'll do the "OnlyEnforceIf" approach for a clearer CP style.
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                # If plane i lands before j:
                model.Add(
                    landing_times[j] >= landing_times[i] + separation_times[i][j]
                ).OnlyEnforceIf(landing_order[(i, j)])
                # If plane j lands before i:
                model.Add(
                    landing_times[i] >= landing_times[j] + separation_times[j][i]
                ).OnlyEnforceIf(landing_order[(i, j)].Not())

    # (d) For each pair (i < j), exactly one ordering:
    #     x[i,j] + x[j,i] = 1
    # Using OnlyEnforceIf, we say x[i,j] is the complement of x[j,i], but let's
    # just do an AddBoolOr constraint or equivalently:
    for i in range(num_planes):
        for j in range(i + 1, num_planes):
            # x[i,j] + x[j,i] = 1 => they can't be both 0 or both 1
            # We'll do it with a simple linear constraint:
            model.Add(landing_order[(i, j)] + landing_order[(j, i)] == 1)

    # (Optional) freeze_time usage would go here, if needed.
    # E.g. if plane i must keep some partial schedule. 
    # We'll skip that unless specifically needed.

    # 3) Objective: Minimize sum of earliness + lateness cost
    # objective = sum( e[i]*earliness[i] + l[i]*lateness[i] )
    objective_terms = []
    for i in range(num_planes):
        objective_terms.append(cost_e[i] * earliness[i])
        objective_terms.append(cost_l[i] * lateness[i])
    model.Minimize(sum(objective_terms))

    # 4) Return the model and the variables in a dictionary
    variables = {
        "landing_times": landing_times,
        "earliness": earliness,
        "lateness": lateness,
        "landing_order": landing_order,
    }

    return model, variables




In [4]:
from ortools.sat.python import cp_model

def solve_cp_model(num_planes, freeze_time, planes_data, separation_times):
    # 1) Build the CP model
    model, variables = create_or_tools_cp_model(
        num_planes, freeze_time, planes_data, separation_times
    )

    # 2) Solve
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # 3) Interpret the results
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("Status:", status)
        print("Optimal Cost:", solver.ObjectiveValue())
        print("\nOptimal Landing Times:")
        landing_times = variables["landing_times"]
        for i in range(num_planes):
            print(
                f"Plane {i+1}: {solver.Value(landing_times[i])} "
                f"- Target Time: {planes_data[i]['target_landing_time']}"
            )
        print("\nLanding Order Variables (in CP, these will be 0 or 1):")
        x = variables["landing_order"]
        for i in range(num_planes):
            for j in range(num_planes):
                if i != j:
                    print(
                        f"LandingOrder_{i}_{j}: {solver.Value(x[(i, j)])}"
                    )

    else:
        print("No optimal solution found.")
        print("Status:", status)


# Example main usage:
if __name__ == "__main__":
    # Suppose you already read data as in your MIP example:
    #   num_planes, freeze_time, planes_data, separation_times
    # (You can reuse the same function read_data_from_file or your custom logic.)
    
    # We'll just show the call:
    if num_planes is not None:
        solve_cp_model(num_planes, freeze_time, planes_data, separation_times)


Status: 4
Optimal Cost: 700.0

Optimal Landing Times:
Plane 1: 165 - Target Time: 155
Plane 2: 258 - Target Time: 258
Plane 3: 98 - Target Time: 98
Plane 4: 106 - Target Time: 106
Plane 5: 118 - Target Time: 123
Plane 6: 134 - Target Time: 135
Plane 7: 126 - Target Time: 138
Plane 8: 142 - Target Time: 140
Plane 9: 150 - Target Time: 150
Plane 10: 180 - Target Time: 180

Landing Order Variables (in CP, these will be 0 or 1):
LandingOrder_0_1: 1
LandingOrder_0_2: 0
LandingOrder_0_3: 0
LandingOrder_0_4: 0
LandingOrder_0_5: 0
LandingOrder_0_6: 0
LandingOrder_0_7: 0
LandingOrder_0_8: 0
LandingOrder_0_9: 1
LandingOrder_1_0: 0
LandingOrder_1_2: 0
LandingOrder_1_3: 0
LandingOrder_1_4: 0
LandingOrder_1_5: 0
LandingOrder_1_6: 0
LandingOrder_1_7: 0
LandingOrder_1_8: 0
LandingOrder_1_9: 0
LandingOrder_2_0: 1
LandingOrder_2_1: 1
LandingOrder_2_3: 1
LandingOrder_2_4: 1
LandingOrder_2_5: 1
LandingOrder_2_6: 1
LandingOrder_2_7: 1
LandingOrder_2_8: 1
LandingOrder_2_9: 1
LandingOrder_3_0: 1
LandingOrde


### **What This Code Does**
1. Reads the data from your file `airland1.txt` (using your `read_data_from_file` function).  
2. Builds the single-runway CP model:
   - Creates integer decision variables `t[i]` for landing times.  
   - Creates continuous/integer variables `E_[i]` (earliness) and `L_[i]` (lateness).  
   - Creates binary variables `x[i,j]` to determine the order between plane *i* and plane *j*.  
   - Enforces time windows, separation constraints, and earliness/lateness definitions.  
   - Minimizes the sum of earliness + lateness penalties.  
3. Solves it using `docplex.cp`.  

> In the output, you’ll see each plane’s landing time (`t_val`) and the values of `E'` and `L'`. If you print out the `x[i,j]` variables, you can also interpret the exact ordering of landings. 

---

### **Optional Enhancements**
- **Freeze Time**: If you want to enforce a partial ordering for planes that appear before `freeze_time`, or any other logic, you can do so by adding constraints involving `freeze_time`.  
- **Use CP Scheduling Features**: Instead of big-M constraints, you could use `mdl.sequence_var()`, `mdl.no_overlap()`, or `interval_var` constructs from `docplex.cp` to handle the scheduling in a more “native” CP manner.  

With this, you have a full end-to-end **CP** solution procedure that is **consistent** with your data and your earlier problem formulation.


# Multiple Runways

<h4 style="color:#0033a0">Parameters</h4>

\begin{align}
p & \quad \text{Number of aircraft} \\
\mathcal{R} & \quad \text{Set of runways (}\{1,\ldots,R\}\text{)} \\
E_i,\,T_i,\,L_i & \quad \text{(as before)} \\
e_i,\,l_i & \quad \text{(earliness/lateness cost coefficients)} \\
S_{i,j} & \quad \text{Separation time needed if } i \text{ and } j \text{ share a runway} \\
\end{align}


<br/> <h4 style="color:#0033a0">Decision Variables</h4>

\begin{align*}
t_i & \quad \text{Landing time of aircraft } i \\
r_i & \quad \text{Runway chosen for aircraft } i \quad (r_i \in \mathcal{R}) \\
E'_i & \quad \text{Earliness of aircraft } i \quad (E'_i \ge 0) \\
L'_i & \quad \text{Lateness of aircraft } i \quad (L'_i \ge 0) \\
x_{i,j} & =
\begin{cases}
1, & \text{if } i \text{ lands before } j \text{ on the same runway},\\
0, & \text{otherwise (different runways or } j \text{ lands first)}
\end{cases}
\end{align*}


<br/> <h4 style="color:#0033a0">Objective Function and Constraints</h4>

\begin{align}
\text{Minimize} \quad
& \sum_{i=1}^p \Bigl( e_i \, E'_i + l_i \, L'_i \Bigr)
&& (1)
\\[6pt]
\text{subject to:} \quad
& E_i \;\le\; t_i \;\le\; L_i
&& \forall i
&& (2)
\\
& r_i \in \mathcal{R}
&& \forall i
&& (3)
\\
& \text{If } r_i = r_j \text{ then } 
   t_j \;\ge\; t_i + S_{i,j} \;-\; M\,(1 - x_{i,j})
&& \forall i\neq j
&& (4)
\\
& \text{If } r_i = r_j \text{ then } 
   t_i \;\ge\; t_j + S_{j,i} \;-\; M\,x_{i,j}
&& \forall i\neq j
&& (5)
\\
& x_{i,j} + x_{j,i} \;\le\; 1 
&& \forall i \neq j
&& (6)
\\
& t_i + E'_i = T_i \;\;(\text{if } t_i \le T_i), 
   \quad
   t_i - L'_i = T_i \;\;(\text{if } t_i \ge T_i)
&& \forall i
&& (7)
\\
& x_{i,j} \in \{0,1\}, \quad E'_i,L'_i \ge 0, \quad t_i \ge 0
&& (8)
\end{align}
